In [1]:
from pan20.fake import inputs
#from pan20.fake import features
from pan20 import fake

import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
#from string import punctuation
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
#data = inputs.parse('data/pan20-author-profiling-training-2020-02-23.zip')

In [3]:
df = fake.load_data()

In [4]:
truth = {x['author']: x['label'] for x in fake.load_truth()}

In [5]:
df['label'] = df.author.apply(lambda x: truth[x])

In [17]:
# class PreProcessTweets:
#     def __init__(self):
#         self._stopwords = set(stopwords.words('english') + list(punctuation) + ['AT_USER','URL'])
        
#     def processTweets(self, list_of_tweets):
#         processedTweets=[]
#         for tweet in list_of_tweets:
#             processedTweets.append((self._processTweet(tweet)))
#         return processedTweets
    
#     def _processTweet(self, tweet):
#         tweet = tweet.lower() # convert text to lower-case
#         tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
#         #tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
#         tweet = re.sub(r'#([^\s]+)', r'', tweet) # remove the # in #hashtag
#         tweet = word_tokenize(tweet) # remove repeated characters (helloooooooo into hello)
#         return [word for word in tweet if word not in self._stopwords]
# tweetProcessor = PreProcessTweets()
# preprocessedTrainingSet = tweetProcessor.processTweets(df['tweet'])

def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def remove_hashtag(string):
    hashtag_pattern = re.compile("#([^\s]+)")
    
    return re.sub(' {2,}', '', hashtag_pattern.sub(r'', string))

In [18]:
df['tweet'] = df['tweet'].apply(remove_emoji)
df['tweet'] = df['tweet'].apply(remove_hashtag)

In [67]:
tfidf = TfidfVectorizer(tokenizer=word_tokenize, analyzer='word', ngram_range=(1,2), stop_words = None, smooth_idf=True, sublinear_tf=True, norm='l2')
#tfidf = TfidfVectorizer(tokenizer=identity_tokenizer, analyzer='word', ngram_range=(1,2), stop_words = None, smooth_idf=True, sublinear_tf=True, norm='l2', lowercase=False, token_pattern='(?u)\\b\\w\\w*\\b')
#tfidf.fit(df[0])
train_df['tweet'] = tfidf.fit_transform(train_df['tweet'])
test_df['tweet'] = tfidf.transform(test_df['tweet'])

In [74]:
for i in range(len(train_df)):
    train_df['tweet'][i] = train_df['tweet'][i].toarray()

MemoryError: Unable to allocate 37.1 GiB for an array with shape (24000, 207281) and data type float64

In [71]:
train_df['tweet'] = train_df['tweet'].apply(toarray())

NameError: name 'toarray' is not defined

In [68]:
test_df['tweet']

50         (0, 203211)\t0.1515247440751767\n  (0, 19860...
98         (0, 203211)\t0.1515247440751767\n  (0, 19860...
37         (0, 203211)\t0.1515247440751767\n  (0, 19860...
62         (0, 203211)\t0.1515247440751767\n  (0, 19860...
1          (0, 203211)\t0.1515247440751767\n  (0, 19860...
95         (0, 203211)\t0.1515247440751767\n  (0, 19860...
70         (0, 203211)\t0.1515247440751767\n  (0, 19860...
97         (0, 203211)\t0.1515247440751767\n  (0, 19860...
45         (0, 203211)\t0.1515247440751767\n  (0, 19860...
6          (0, 203211)\t0.1515247440751767\n  (0, 19860...
94         (0, 203211)\t0.1515247440751767\n  (0, 19860...
68         (0, 203211)\t0.1515247440751767\n  (0, 19860...
53         (0, 203211)\t0.1515247440751767\n  (0, 19860...
91         (0, 203211)\t0.1515247440751767\n  (0, 19860...
89         (0, 203211)\t0.1515247440751767\n  (0, 19860...
55         (0, 203211)\t0.1515247440751767\n  (0, 19860...
96         (0, 203211)\t0.1515247440751767\n  (0, 19860.

In [144]:
# x_feats = [c for c in df.columns if c not in ['author', 'label']]

In [9]:
#x_train = train[x_feats].values
#x_test = test[x_feats].values
y_train= train['label'].values
#y_test = test['label'].values
y_test = df.drop(train.index)['label'].values

In [9]:
from sklearn.naive_bayes import GaussianNB

In [10]:
clf = GaussianNB()

In [12]:
clf.fit(x_train_tfidf.toarray(), y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
predictions = clf.predict(x_test_tfidf.toarray())

In [177]:
## using len(data)=10000

np.mean(predictions == y_test)

0.7515

In [189]:
## using len(data)=15000

np.mean(predictions == y_test)

0.731

In [24]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

clf = Pipeline([('vect', TfidfVectorizer()),
                ('funct', FunctionTransformer(lambda x: x.todense(), accept_sparse=True)),
                ('gnb', GaussianNB())])

scores = cross_val_score(clf, df['tweet'].values, df['label'], cv=5)
print(scores)
print(np.mean(scores))

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_function_transformer.py:97: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_function_transformer.py:97: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_function_transformer.py:97: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_function_transformer.py:97: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_function_tra

[0.545      0.54433333 0.52083333 0.58833333 0.52183333]
0.5440666666666667


NameError: name 'X' is not defined